<a href="https://colab.research.google.com/github/SanjayMSD/Data-structures-and-Algorithms-/blob/master/1_Get_Emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os

df = pd.read_csv("filtered_companies_100 ROWS.csv")
print("Loaded", len(df), "rows")



NameError: name 'BASE_DIR' is not defined